In [ ]:
from pymatgen.core import Structure
from workflow_utils import *
from distortion_analysis import *

## Extract modes info (irreps, symmetry)

In [ ]:
subdir = 'data/phon'
irreps_file = 'irreps_std.yaml'
df = parse_irreps(subdir, irreps_file, dim_str='2 2 2', kpt='0 0 0')
df

## Generate modulated structures

In [ ]:
subdir = 'data/phon'
irreps_file = 'irreps_std.yaml'

### Define the dimensions of the modulation
### NB: The gen_mod_strucs() function converts to primitive so these dimensions can be larger than needed but they could be, for example 1,1,1 if you're only modulating G-pt modes
moddims = [2,2,2]

### Detail the kpoints, indices, and displacements of modes you want to modulate 
label_ls = [['G'],['G'],['X'],['Z'],['Z'],['G','G'],['Z','Z']]
indices_ls = [[1],[2],[1],[1],[2],[1,2],[1,2]]
disps_ls = [[1 for label in labels] for labels in label_ls]

### Syntactic sugar for defining the kpoints/labels for this structure
kpts_dict = {'G':'0 0 0', 'X':'0.5 0 0', 'Z':'0.5 0.5 -0.5'}
kpts_ls = [[kpts_dict[label] for label in labels] for labels in label_ls]

### Detail the locations of your phonon calculation directories
phon_dirs = glob.glob('data/phon')
print(phon_dirs)
phon_dirs = [os.path.abspath(p) for p in phon_dirs]

### For each phonon directory, generate the modulated structures
for phon_dir in phon_dirs:
    ### Modulate structures for each desired mode
    for i in range(len(label_ls)):
        struc, path = gen_mod_struc(phon_dir, label_ls[i], kpts_ls[i], indices_ls[i], disps_ls[i], moddims)
        print(path)
        print(struc)
        os.makedirs(f'{path}/processed_strucs', exist_ok=True)
        struc.to(f'{path}/processed_strucs/MPOSCAR.vasp', 'poscar')

## Generate interpolations between distorted structures


In [ ]:
subdir = 'data/phon'
irreps_file = 'irreps_std.yaml'

### Define the dimensions of the modulation
### NB: The gen_mod_strucs() function converts to primitive so these dimensions can be larger than needed but they could be, for example 1,1,1 if you're only modulating G-pt modes
moddims = [2,2,2]

### Detail the kpoints, indices, and displacements of modes you want to modulate 
label_ls = [['G'],['G'],['X'],['Z'],['Z'],['G','G'],['Z','Z']]
indices_ls = [[1],[2],[1],[1],[2],[1,2],[1,2]]
disps_ls = [[1 for label in labels] for labels in label_ls]

### Syntactic sugar for defining the kpoints/labels for this structure
kpts_dict = {'G':'0 0 0', 'X':'0.5 0 0', 'Z':'0.5 0.5 -0.5'}
kpts_ls = [[kpts_dict[label] for label in labels] for labels in label_ls]

### Detail the locations of your phonon calculation directories
phon_dirs = glob.glob('data/phon')
print(phon_dirs)
phon_dirs = [os.path.abspath(p) for p in phon_dirs]

### For each phonon directory, generate the modulated structures
for phon_dir in phon_dirs:
    ### Modulate structures for each desired mode
    for i in range(len(label_ls)):
        struc, path = gen_mod_struc(phon_dir, label_ls[i], kpts_ls[i], indices_ls[i], disps_ls[i], moddims)
        print(path)
        print(struc)
        os.makedirs(f'{path}/processed_strucs', exist_ok=True)
        struc.to(f'{path}/processed_strucs/MPOSCAR.vasp', 'poscar')

## Generate a -P-distorted structure from a centrosymmetric and +P-distorted structure

In [ ]:
### Import and sortstructures

centro = Structure.from_file('data/struc_centro.vasp')
centro = sort_sites(centro)
print(centro)

p = Structure.from_file('data/struc_plusP.vasp')
p = sort_sites(p)
print(p)

In [ ]:
### Examine difference between structures to check for errors
print('+P distortions')
print(centro.frac_coords - p.frac_coords)

In [ ]:
### Generate the mirror image, check that differences are opposite of plusP
_, minusP = centro.interpolate(p, end_amplitude=-1, nimages=1)
print('-P distortions')
print(centro.frac_coords - minusP.frac_coords)
print('\n-P + +P = 0')
print((centro.frac_coords - minusP.frac_coords) + (centro.frac_coords - p.frac_coords))
### Save the -P structure
minusP.to_file(filename='data/struc_minusP.vasp', fmt='poscar')